In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./model.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.moogparts.com/driv/partfinder/api.catalog.partslist?brand=fllj&locale=en_US&country_code=US&year_id=' + input_.loc[a, 'Year_Id'] + '&make_id=' + input_.loc[a, 'Make_Id'] + '&model_id=' + input_.loc[a, 'Model_Id'] + '&vehicle_group_ids=2',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =

                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_part = json_data['application_list']['applications']
                
                # = = = = = = = = = = = = = = =
                
                list_position = [part['position']['value'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_drive = [part['drive']['value'].strip() if 'drive' in part else '' for part in list_part]

                # = = = = = = = = = = = = = = =

                list_type_ = [part['category']['part_type_value'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_subcategory = [part['category']['sub_category_value'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_category = [part['category']['category_value'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_product = [part['product'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_brand = [part['brand'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_qty = [part['quantity_per_application'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_part_number = [part['part_number'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_part_name = [part['part_name'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_fit = [part['fits'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_criteria = [part['additional_fit_criteria'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_src = ['https://www.drivparts.com' + part['dam_assets']['productPrimaries'][0]['url'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                list_url = [part['part_detail_redirect_url'].strip() for part in list_part]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Year_Id': int(input_.loc[a, 'Year_Id']),
                                        'Year': input_.loc[a, 'Year'],
                                        'Make_Id': int(input_.loc[a, 'Make_Id']),
                                        'Make': input_.loc[a, 'Make'],
                                        'Model_Id': int(input_.loc[a, 'Model_Id']),
                                        'Model': input_.loc[a, 'Model'],
                                        'Part_No.': [i+1 for i in range(len(list_part))],
                                        'Drive': list_drive,
                                        'Position': list_position,
                                        'Category': list_category,
                                        'Subcategory': list_subcategory,
                                        'Type': list_type_,
                                        'Product': list_product,
                                        'Part_Number': list_part_number,
                                        'Part_Name': list_part_name,
                                        'Brand': list_brand,
                                        'Qty': list_qty,
                                        'Fit': list_fit,
                                        'Criteria': list_criteria,
                                        'Pic': '',
                                        'Url': list_url,
                                        'Src': list_src,
                                        'JOIN_PNB': [part_number + ';' + brand for part_number, brand in zip(list_part_number, list_brand)]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Year_Id': int(input_.loc[a, 'Year_Id']),
                                     'Year': input_.loc[a, 'Year'],
                                     'Make_Id': int(input_.loc[a, 'Make_Id']),
                                     'Make': input_.loc[a, 'Make'],
                                     'Model_Id': int(input_.loc[a, 'Model_Id']),
                                     'Model': input_.loc[a, 'Model']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Year'] + ' - ' + input_.loc[a, 'Make'] + ' - ' + input_.loc[a, 'Model'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Year_Id', 'Make_Id', 'Model_Id', 'Part_No.'], ascending=[False, True, True, True]).reset_index(drop=True)
output_correct.to_excel('./menu-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Year_Id', 'Make_Id', 'Model_Id'], ascending=[False, True, True]).reset_index(drop=True)
    output_error.to_excel('./menu_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：8988

[ok] - 2023 - Hyundai - Kona
[尝试次数：1] - [剩余数量：8978] - [当前时间：10:14:09]

[ok] - 2023 - Hyundai - Santa Fe
[尝试次数：1] - [剩余数量：8977] - [当前时间：10:14:09]

[ok] - 2023 - Hyundai - Tucson
[尝试次数：1] - [剩余数量：8976] - [当前时间：10:14:09]

[ok] - 2023 - Hyundai - Nexo
[尝试次数：1] - [剩余数量：8975] - [当前时间：10:14:10]

[ok] - 2024 - Nissan - Rogue
[尝试次数：1] - [剩余数量：8974] - [当前时间：10:14:10]

[ok] - 2023 - Hyundai - Santa Cruz
[尝试次数：1] - [剩余数量：8973] - [当前时间：10:14:10]

[ok] - 2023 - Hyundai - Elantra N
[尝试次数：1] - [剩余数量：8972] - [当前时间：10:14:10]

[ok] - 2023 - Kia - Telluride
[尝试次数：1] - [剩余数量：8971] - [当前时间：10:14:11]

[ok] - 2023 - Jaguar - F-Type
[尝试次数：1] - [剩余数量：8970] - [当前时间：10:14:11]

[ok] - 2023 - Volvo - S60
[尝试次数：1] - [剩余数量：8969] - [当前时间：10:14:11]

[ok] - 2023 - Volvo - S90
[尝试次数：1] - [剩余数量：8968] - [当前时间：10:14:11]

[ok] - 2023 - Hyundai - Palisade
[尝试次数：1] - [剩余数量：8967] - [当前时间：10:14:12]

[ok] - 2023 - Volvo - XC90
[尝试次数：1] - [剩余数量：8966] - [当前时间：10:14:12]

[ok] - 2023 - Hyundai - Kona N
[尝试次数：1] - [剩余数量：8965

[ok] - 2022 - Hyundai - Santa Cruz
[尝试次数：1] - [剩余数量：8865] - [当前时间：10:14:39]

[ok] - 2022 - Hyundai - Palisade
[尝试次数：1] - [剩余数量：8864] - [当前时间：10:14:40]

[ok] - 2022 - Jaguar - F-Type
[尝试次数：1] - [剩余数量：8863] - [当前时间：10:14:40]

[ok] - 2022 - Kia - Forte
[尝试次数：1] - [剩余数量：8862] - [当前时间：10:14:40]

[ok] - 2022 - Kia - Sorento
[尝试次数：1] - [剩余数量：8861] - [当前时间：10:14:40]

[ok] - 2022 - Kia - Sportage
[尝试次数：1] - [剩余数量：8860] - [当前时间：10:14:40]

[ok] - 2022 - Kia - K5
[尝试次数：1] - [剩余数量：8859] - [当前时间：10:14:41]

[ok] - 2022 - Kia - Seltos
[尝试次数：1] - [剩余数量：8858] - [当前时间：10:14:41]

[ok] - 2022 - Kia - Telluride
[尝试次数：1] - [剩余数量：8857] - [当前时间：10:14:41]

[ok] - 2022 - Kia - Carnival
[尝试次数：1] - [剩余数量：8856] - [当前时间：10:14:41]

[ok] - 2022 - Kia - Niro
[尝试次数：1] - [剩余数量：8855] - [当前时间：10:14:41]

[ok] - 2022 - Hyundai - Elantra N
[尝试次数：1] - [剩余数量：8854] - [当前时间：10:14:41]

[ok] - 2022 - Volvo - S60
[尝试次数：1] - [剩余数量：8853] - [当前时间：10:14:41]

[ok] - 2022 - Volvo - S90
[尝试次数：1] - [剩余数量：8852] - [当前时间：10:14:42]

[ok] - 2022

[ok] - 2022 - Audi - SQ5 Sportback
[尝试次数：1] - [剩余数量：8752] - [当前时间：10:15:11]

[ok] - 2022 - Audi - RS7 Sportback
[尝试次数：1] - [剩余数量：8751] - [当前时间：10:15:11]

[ok] - 2022 - Volkswagen - Passat
[尝试次数：1] - [剩余数量：8750] - [当前时间：10:15:11]

[ok] - 2022 - Audi - Q5 Sportback
[尝试次数：1] - [剩余数量：8749] - [当前时间：10:15:12]

[ok] - 2022 - Volkswagen - Tiguan
[尝试次数：1] - [剩余数量：8748] - [当前时间：10:15:12]

[ok] - 2022 - Volkswagen - Golf R
[尝试次数：1] - [剩余数量：8747] - [当前时间：10:15:12]

[ok] - 2022 - Volkswagen - Taos
[尝试次数：1] - [剩余数量：8746] - [当前时间：10:15:13]

[ok] - 2022 - Volkswagen - Jetta
[尝试次数：1] - [剩余数量：8745] - [当前时间：10:15:13]

[ok] - 2022 - Volkswagen - Arteon
[尝试次数：1] - [剩余数量：8744] - [当前时间：10:15:13]

[ok] - 2022 - Audi - SQ7
[尝试次数：1] - [剩余数量：8743] - [当前时间：10:15:14]

[ok] - 2022 - Lexus - ES350
[尝试次数：1] - [剩余数量：8742] - [当前时间：10:15:14]

[ok] - 2022 - Lexus - RX350
[尝试次数：1] - [剩余数量：8741] - [当前时间：10:15:14]

[ok] - 2022 - Lexus - ES300h
[尝试次数：1] - [剩余数量：8740] - [当前时间：10:15:15]

[ok] - 2022 - Lexus - RX450h
[尝试次数：1] -

[ok] - 2021 - Buick - Encore
[尝试次数：1] - [剩余数量：8639] - [当前时间：10:15:45]

[ok] - 2021 - Jeep - Gladiator
[尝试次数：1] - [剩余数量：8638] - [当前时间：10:15:45]

[ok] - 2021 - Buick - Encore GX
[尝试次数：1] - [剩余数量：8637] - [当前时间：10:15:46]

[ok] - 2021 - Cadillac - Escalade
[尝试次数：1] - [剩余数量：8636] - [当前时间：10:15:46]

[ok] - 2021 - Cadillac - XT5
[尝试次数：1] - [剩余数量：8635] - [当前时间：10:15:46]

[ok] - 2023 - Ford - Edge
[尝试次数：2] - [剩余数量：8634] - [当前时间：10:15:46]

[ok] - 2021 - Cadillac - XT6
[尝试次数：1] - [剩余数量：8633] - [当前时间：10:15:46]

[ok] - 2021 - Chevrolet - Camaro
[尝试次数：1] - [剩余数量：8632] - [当前时间：10:15:47]

[ok] - 2021 - Cadillac - Escalade ESV
[尝试次数：1] - [剩余数量：8631] - [当前时间：10:15:47]

[ok] - 2021 - Chevrolet - Express 2500
[尝试次数：1] - [剩余数量：8630] - [当前时间：10:15:47]

[ok] - 2021 - Chevrolet - Malibu
[尝试次数：1] - [剩余数量：8629] - [当前时间：10:15:47]

[ok] - 2021 - Chevrolet - Trailblazer
[尝试次数：1] - [剩余数量：8628] - [当前时间：10:15:47]

[ok] - 2021 - Chevrolet - Express 3500
[尝试次数：1] - [剩余数量：8627] - [当前时间：10:15:48]

[ok] - 2021 - Chevrolet 

[ok] - 2021 - Lexus - RC F
[尝试次数：1] - [剩余数量：8525] - [当前时间：10:16:09]

[ok] - 2021 - Lexus - LC500
[尝试次数：1] - [剩余数量：8524] - [当前时间：10:16:10]

[ok] - 2021 - Lexus - RC300
[尝试次数：1] - [剩余数量：8523] - [当前时间：10:16:10]

[ok] - 2021 - Lexus - LS500
[尝试次数：1] - [剩余数量：8522] - [当前时间：10:16:10]

[ok] - 2021 - Lexus - NX300
[尝试次数：1] - [剩余数量：8521] - [当前时间：10:16:10]

[ok] - 2021 - Lexus - RX350L
[尝试次数：1] - [剩余数量：8520] - [当前时间：10:16:10]

[ok] - 2021 - Lexus - RX450hL
[尝试次数：1] - [剩余数量：8519] - [当前时间：10:16:11]

[ok] - 2021 - Lexus - LC500h
[尝试次数：1] - [剩余数量：8518] - [当前时间：10:16:11]

[ok] - 2021 - Lexus - LS500h
[尝试次数：1] - [剩余数量：8517] - [当前时间：10:16:11]

[ok] - 2021 - Toyota - Highlander
[尝试次数：1] - [剩余数量：8516] - [当前时间：10:16:11]

[ok] - 2021 - Toyota - 4Runner
[尝试次数：1] - [剩余数量：8515] - [当前时间：10:16:11]

[ok] - 2021 - Toyota - Prius
[尝试次数：1] - [剩余数量：8514] - [当前时间：10:16:12]

[ok] - 2021 - Toyota - RAV4
[尝试次数：1] - [剩余数量：8513] - [当前时间：10:16:12]

[ok] - 2021 - Toyota - Corolla
[尝试次数：1] - [剩余数量：8512] - [当前时间：10:16:12]

[ok

...

[ok] - 1988 - Dodge - Dakota
[尝试次数：1] - [剩余数量：457] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Aries
[尝试次数：1] - [剩余数量：456] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Dynasty
[尝试次数：1] - [剩余数量：455] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Daytona
[尝试次数：1] - [剩余数量：454] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - 600
[尝试次数：1] - [剩余数量：453] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Caravan
[尝试次数：1] - [剩余数量：452] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Lancer
[尝试次数：1] - [剩余数量：451] - [当前时间：10:48:54]

[ok] - 1988 - Dodge - Grand Caravan
[尝试次数：1] - [剩余数量：450] - [当前时间：10:48:55]

[ok] - 1988 - Dodge - Mini Ram
[尝试次数：1] - [剩余数量：449] - [当前时间：10:48:55]

[ok] - 1988 - Dodge - Omni
[尝试次数：1] - [剩余数量：448] - [当前时间：10:48:55]

[ok] - 1988 - Dodge - Shadow
[尝试次数：1] - [剩余数量：447] - [当前时间：10:48:55]

[ok] - 1988 - Jeep - Cherokee
[尝试次数：1] - [剩余数量：446] - [当前时间：10:48:56]

[ok] - 1988 - Jeep - Wagoneer
[尝试次数：1] - [剩余数量：445] - [当前时间：10:48:56]

[ok] - 1988 - Plymouth - Grand Voyager
[尝试次数：1] - [剩余数量：444] - [当前时间：10:48:56]

[o

[ok] - 1987 - Plymouth - Horizon
[尝试次数：1] - [剩余数量：342] - [当前时间：10:49:18]

[ok] - 1987 - Buick - Electra
[尝试次数：1] - [剩余数量：341] - [当前时间：10:49:19]

[ok] - 1987 - Plymouth - Expo
[尝试次数：1] - [剩余数量：340] - [当前时间：10:49:19]

[ok] - 1987 - Plymouth - Caravelle
[尝试次数：1] - [剩余数量：339] - [当前时间：10:49:19]

[ok] - 1987 - Buick - LeSabre
[尝试次数：1] - [剩余数量：338] - [当前时间：10:49:20]

[ok] - 1987 - Buick - Century
[尝试次数：1] - [剩余数量：337] - [当前时间：10:49:20]

[ok] - 1987 - Buick - Riviera
[尝试次数：1] - [剩余数量：336] - [当前时间：10:49:20]

[ok] - 1987 - Buick - Skyhawk
[尝试次数：1] - [剩余数量：335] - [当前时间：10:49:20]

[ok] - 1987 - Plymouth - Sundance
[尝试次数：1] - [剩余数量：334] - [当前时间：10:49:21]

[ok] - 1987 - Cadillac - Allante
[尝试次数：1] - [剩余数量：333] - [当前时间：10:49:21]

[ok] - 1987 - Cadillac - Commercial Chassis
[尝试次数：1] - [剩余数量：332] - [当前时间：10:49:21]

[ok] - 1987 - Buick - Somerset
[尝试次数：1] - [剩余数量：331] - [当前时间：10:49:21]

[ok] - 1987 - Cadillac - Cimarron
[尝试次数：1] - [剩余数量：330] - [当前时间：10:49:22]

[ok] - 1987 - Plymouth - Turismo
[尝试次数：1] -

[ok] - 1986 - Ford - EXP
[尝试次数：1] - [剩余数量：228] - [当前时间：10:49:44]

[ok] - 1986 - Ford - Escort
[尝试次数：1] - [剩余数量：227] - [当前时间：10:49:44]

[ok] - 1986 - Ford - Tempo
[尝试次数：1] - [剩余数量：226] - [当前时间：10:49:44]

[ok] - 1986 - Ford - Taurus
[尝试次数：1] - [剩余数量：225] - [当前时间：10:49:45]

[ok] - 1986 - Buick - Century
[尝试次数：1] - [剩余数量：224] - [当前时间：10:49:45]

[ok] - 1986 - Mercury - Sable
[尝试次数：1] - [剩余数量：223] - [当前时间：10:49:45]

[ok] - 1986 - Mercury - Topaz
[尝试次数：1] - [剩余数量：222] - [当前时间：10:49:46]

[ok] - 1986 - Acura - Integra
[尝试次数：1] - [剩余数量：221] - [当前时间：10:49:46]

[ok] - 1986 - Honda - Civic
[尝试次数：1] - [剩余数量：220] - [当前时间：10:49:46]

[ok] - 1986 - Mercury - Lynx
[尝试次数：1] - [剩余数量：219] - [当前时间：10:49:46]

[ok] - 1986 - Honda - Accord
[尝试次数：1] - [剩余数量：218] - [当前时间：10:49:46]

[ok] - 1986 - Honda - Wagovan
[尝试次数：1] - [剩余数量：217] - [当前时间：10:49:47]

[ok] - 1986 - Toyota - Camry
[尝试次数：1] - [剩余数量：216] - [当前时间：10:49:47]

[ok] - 1985 - BMW - 318i
[尝试次数：1] - [剩余数量：215] - [当前时间：10:49:47]

[ok] - 1986 - Toyota - MR2
[

[ok] - 1984 - Chevrolet - Corvette
[尝试次数：1] - [剩余数量：114] - [当前时间：10:50:16]

[ok] - 1986 - Saab - 9000
[尝试次数：1] - [剩余数量：113] - [当前时间：10:50:17]

[ok] - 1984 - Chevrolet - S10 Blazer
[尝试次数：1] - [剩余数量：112] - [当前时间：10:50:17]

[ok] - 1984 - GMC - S15 Jimmy
[尝试次数：1] - [剩余数量：111] - [当前时间：10:50:17]

[ok] - 1984 - Chevrolet - S10
[尝试次数：1] - [剩余数量：110] - [当前时间：10:50:17]

[ok] - 1984 - Oldsmobile - Cutlass Ciera
[尝试次数：1] - [剩余数量：109] - [当前时间：10:50:18]

[ok] - 1984 - GMC - S15
[尝试次数：1] - [剩余数量：108] - [当前时间：10:50:18]

[ok] - 1984 - Oldsmobile - Omega
[尝试次数：1] - [剩余数量：107] - [当前时间：10:50:18]

[ok] - 1984 - Pontiac - Fiero
[尝试次数：1] - [剩余数量：106] - [当前时间：10:50:19]

[ok] - 1984 - Ford - EXP
[尝试次数：1] - [剩余数量：105] - [当前时间：10:50:19]

[ok] - 1984 - Oldsmobile - Toronado
[尝试次数：1] - [剩余数量：104] - [当前时间：10:50:19]

[ok] - 1984 - Pontiac - Phoenix
[尝试次数：1] - [剩余数量：103] - [当前时间：10:50:20]

[ok] - 1984 - Pontiac - 6000
[尝试次数：1] - [剩余数量：102] - [当前时间：10:50:20]

[ok] - 1984 - Oldsmobile - Firenza
[尝试次数：1] - [剩余数量：101] - 

[ok] - 1980 - Pontiac - Phoenix
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:42]

[ok] - 1979 - Buick - Riviera
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:42]

[ok] - 1981 - Cadillac - Seville
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:42]

[ok] - 1979 - Cadillac - Eldorado
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:42]

[ok] - 1978 - Plymouth - Horizon
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:43]

[ok] - 1978 - Dodge - Omni
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:43]

[ok] - 1979 - Oldsmobile - Toronado
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:44]

[ok] - 1981 - Plymouth - Reliant
[尝试次数：1] - [剩余数量：0] - [当前时间：10:50:54]

输出ing...

Done ~
